In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
# import argparse

# from models import *
# from utils import progress_bar

In [ ]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

In [ ]:
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='../data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='../data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
'''LeNet in PyTorch.'''
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        out = F.tanh(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.tanh(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.tanh(self.fc1(out))
        out = F.tanh(self.fc2(out))
        out = self.fc3(out)
        return out

# 5*5 필터 6개 -> 28*28 feature map 6개
# sub-sampling -> 14*14 feature map 6개
# 5*5 필터 16개 -> 10*10 feature map 16개
# sub-sampling -> 5*5 feature map 16개
# 5*5 필터 16개 -> 1*1 feature map 120개
# fully connected layer 120 -> 84
# fully connected layer 84 -> 10

## feature map -> flatten -> classifier

In [ ]:
net = LeNet()

In [ ]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        print( 'Loss: %.3f | Acc: %.3f%% (%d/%d)'% (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

#         progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
#                      % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

#             progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
#                          % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
        # Save checkpoint.
    acc = 100.*correct/total
    print(acc)
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [ ]:
for epoch in range(start_epoch, start_epoch+50):
    train(epoch)
    test(epoch)
    scheduler.step()


Epoch: 0
Loss: 2.294 | Acc: 14.062% (18/128)
Loss: 2.291 | Acc: 13.281% (34/256)
Loss: 2.291 | Acc: 13.021% (50/384)
Loss: 2.291 | Acc: 13.867% (71/512)
Loss: 2.290 | Acc: 13.281% (85/640)
Loss: 2.285 | Acc: 13.151% (101/768)


/home/lililio0000/.conda/envs/cv_ez/lib/python3.8/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Loss: 2.276 | Acc: 13.616% (122/896)
Loss: 2.273 | Acc: 14.453% (148/1024)
Loss: 2.265 | Acc: 15.365% (177/1152)
Loss: 2.250 | Acc: 15.781% (202/1280)
Loss: 2.243 | Acc: 16.264% (229/1408)
Loss: 2.229 | Acc: 16.667% (256/1536)
Loss: 2.218 | Acc: 17.127% (285/1664)
Loss: 2.206 | Acc: 17.913% (321/1792)
Loss: 2.198 | Acc: 18.646% (358/1920)
Loss: 2.189 | Acc: 18.750% (384/2048)
Loss: 2.176 | Acc: 19.301% (420/2176)
Loss: 2.168 | Acc: 19.444% (448/2304)
Loss: 2.161 | Acc: 19.655% (478/2432)
Loss: 2.157 | Acc: 19.961% (511/2560)
Loss: 2.151 | Acc: 20.238% (544/2688)
Loss: 2.146 | Acc: 20.455% (576/2816)
Loss: 2.137 | Acc: 20.720% (610/2944)
Loss: 2.130 | Acc: 20.736% (637/3072)
Loss: 2.119 | Acc: 21.125% (676/3200)
Loss: 2.116 | Acc: 21.304% (709/3328)
Loss: 2.112 | Acc: 21.499% (743/3456)
Loss: 2.109 | Acc: 21.624% (775/3584)
Loss: 2.100 | Acc: 21.983% (816/3712)
Loss: 2.098 | Acc: 22.109% (849/3840)
Loss: 2.094 | Acc: 22.379% (888/3968)
Loss: 2.091 | Acc: 22.681% (929/4096)
Loss: 2.090 |

In [ ]:
# torch.load(state, './checkpoint/ckpt.pth')